<a href="https://colab.research.google.com/github/rahiakela/building-computer-vision-applications-using-artificial-neural-networks/blob/master/6-deep-learning-in-object-detection/1_training_object_detection_model_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Object Detection Model Using TensorFlow

Object detection models are very compute-intensive and require both a lot of memory and a powerful processor. Most general-purpose laptops or computers may not be able to handle the computations necessary to build and train an object detection model.

Thankfully, Google provides a limited amount of GPU-based computing
for free. It has been proven that these models run many folds faster on a GPU than on a CPU. Therefore, it is important to learn how to train a model on a GPU.

Let’s first define what our learning objective is and how we want to achieve it.

- **Objective**: Learn how to train an object detection model using Keras
and TensorFlow.
- **Dataset**: The Oxford-IIIT Pet dataset, which is freely available at
robots.ox.ac.uk/~vgg/data/pets/. The dataset consists of 37
categories of pets with roughly 200 images for each class. The images
have large variations in scale, pose, and lighting. They are already
annotated with bounding boxes and labeled.

- **Important note**: At the time of writing this book, TensorFlow version 2
does not support the training of custom models for object detection.
Therefore, we will use TensorFlow version 1.15 to train the model.
The TensorFlow team and the open source community are working to migrate the version 1 code to support the training of custom object
detection models in version 2.

## Setup

Google Colab is an interactive programming environment and does not give direct
access to the underlying operating system. You can invoke the shell using `%%shell`, which remains active within a single block of code cells it is invoked from. You can invoke the shell from as many code blocks as needed.

To set up our environment, we will follow the following steps:

1. Install the necessary libraries needed to execute our TensorFlow
code and train our model.

In [ ]:
# Installing the Necessary Libraries and Packages
%%shell
%tensorflow_version 1.x
sudo apt-get install protobuf-compiler python-pil python-lxml python-tk
pip install --user Cython
pip install --user contextlib2
pip install --user pillow
pip install --user lxml
pip install --user matplotlib

2. Download the TensorFlow “models” project from the GitHub repository, and build and install it in your working environment.

In [ ]:
# Downloading the TensorFlow Models Project, Building It, and Setting It Up
%%shell
mkdir computer_vision
cd computer_vision
git clone https://github.com/ansarisam/models.git
# Official repository
# git clone https://github.com/tensorflow/models.git

# The research directory contains a large number of models that are created and maintained by researchers and not officially supported yet.
cd models/research

# builds the object detection–related source code using the Protobuf compiler.
protoc object_detection/protos/*.proto --python_out=.

# set the PYTHONPATH environment variable to the research and research/slim directories.
export PYTHONPATH=$PYTHONPATH:/content/computer_vision/models/research
export PYTHONPATH=$PYTHONPATH:/content/computer_vision/models/research/slim

# executes a build command using setup.py
python setup.py build
# installs the object detection models in our working environment.
python setup.py install

## Downloading the Oxford-IIIT Pet Dataset